In [702]:
from z3 import *
class f2Poly:
    def __init__(self, terms=None):
        self.terms = set() if terms is None else terms
    def __add__(self, other):
        return f2Poly(self.terms ^ other.terms)
    def __mul__(self, other):
        terms1 = self.terms
        # 情况1：other 是数字（int）
        if isinstance(other, int):
            scalar = other % 2  # 模 2 到 F₂
            if scalar == 0:
                return f2Poly(set())  # 0 * p = 0
            else:
                return self           # 1 * p = p
        # 情况2：other 是 f2Poly
        terms2 = other.terms
        if not terms1 or not terms2:
            return f2Poly(set())  # 0 * anything = 0
        result = set()
        _add = result.add
        _remove = result.remove
        _in = result.__contains__
        for (i1, j1) in terms1:
            for (i2, j2) in terms2:
                term = (i1 + i2, j1 + j2)
                if _in(term):
                    _remove(term)
                else:
                    _add(term)
        return f2Poly(result)
    def __pow__(self, n):
        if n == 0: return f2Poly({(0, 0)})
        if len(self.terms) == 0:  # 零多项式
            if n > 0: return f2Poly(set())
            else: raise ValueError("0 的负幂次无定义")
        if len(self.terms) == 1:  # 单项式
            (i, j) = next(iter(self.terms))
            return f2Poly({(i * n, j * n)})
        # 多项式：快速幂
        if n < 0: raise ValueError("不可逆")
        elif n == 1: return self
        elif n % 2 == 0: return (self * self) ** (n // 2)
        else: return self * (self ** (n - 1))
    def degrees(self):
        terms = self.terms  # 缓存引用
        if not terms:
            return (0, 0)
        max_x = max_y = None
        for m in terms:  # 直接迭代，避免 tuple unpack 暂时开销（微乎其微）
            i, j = m
            if max_x is None or i > max_x:
                max_x = i
            if max_y is None or j > max_y:
                max_y = j
        return (max_x, max_y)
    def mindegrees(self):
        terms = self.terms  # 缓存引用
        if not terms:
            return (0, 0)
        max_x = max_y = None
        for m in terms:  # 直接迭代，避免 tuple unpack 暂时开销（微乎其微）
            i, j = m
            if max_x is None or i < max_x:
                max_x = i
            if max_y is None or j < max_y:
                max_y = j
        return (max_x, max_y)
    def __repr__(self):
        if not self.terms:
            return "0"
        return " + ".join(
            f"x^{i}y^{j}" if i and j else
            f"x^{i}" if i else
            f"y^{j}" if j else
            "1"
            for i, j in self.terms
        )
def antimap(poly):
    inverted_terms = {( -i, -j ) for (i, j) in poly.terms}
    return f2Poly(inverted_terms)
def commute(a, b):
    # 预提取输入
    a0, a1, a2, a3 = a[0][0], a[1][0], a[2][0], a[3][0]
    b0, b1, b2, b3 = b[0][0], b[1][0], b[2][0], b[3][0]
    # 局部引用加速
    _antimap = antimap
    _f2Poly = f2Poly
    result = _f2Poly()
    result += _antimap(a0) * b2
    result += _antimap(a1) * b3
    result += _antimap(a2) * b0
    result += _antimap(a3) * b1
    return result
def excimap(*A, oper):
    result = []
    for a in A:
        # 计算 commute(a, oper)
        excitation = commute(a, oper)
        result.append(excitation)
    return result  # 返回 [F2Poly, F2Poly, ...]，就是一个行向量
def tdmap(f, m):
    shifts = [(i, j) for j in range(-m, m + 1) for i in range(-m, m + 1)]
    f_terms_list = [poly.terms for poly in f]
    result = [None] * len(shifts)
    for idx, (i, j) in enumerate(shifts):
        shifted_f = []
        for terms in f_terms_list:
            shifted_terms = {(i + di, j + dj) for (di, dj) in terms}
            shifted_f.append(f2Poly(shifted_terms))
        result[idx] = shifted_f
    return result
def shape(obj):
    if not obj:
        return (0,)  # 空对象
    if isinstance(obj[0], (list, tuple)):
        # 是矩阵：obj = [ [...], [...] ]
        rows = len(obj)
        cols = len(obj[0])
        return (rows, cols)
    else:
        # 是向量：obj = [a, b, c, d]
        return (len(obj),)
def hstack(A,B):
    if len(A) != len(B):
        raise ValueError("矩阵行数必须相同")
    result = []
    for i in range(len(A)):
        row = A[i] + B[i]  # 直接拼接
        result.append(row)
    return result
def I(n):
    zero = f2Poly()
    one  = f2Poly({(0,0)})
    return [[one if i == j else zero for j in range(n)] for i in range(n)]
def split(augmented):
    if not augmented:
        return [], []
    m = len(augmented)
    total_cols = len(augmented[0])
    n = total_cols - m
    if n < 0:
        raise ValueError("矩阵列数小于行数，无法分割")
    left = []   # 原矩阵部分 A'
    right = []  # 单位阵部分 B'
    for row in augmented:
        left.append(row[:n])   # 前 n 列
        right.append(row[n:])  # 后 m 列
    return left, right
def t(matrix):
    if not matrix or not matrix[0]:
        return []
    return [list(row) for row in zip(*matrix)]
def mulmatrix(A, B):
    if not A or not B:
        return []
    m = len(A)
    n = len(A[0])
    if len(B) != n:
        raise ValueError(f"矩阵维度不匹配：A 是 {m}x{n}，B 是 {len(B)}x{len(B[0])}")
    p = len(B[0])
    zero = f2Poly()
    C = [[zero for _ in range(p)] for _ in range(m)]
    for i in range(m):
        for j in range(p):
            total = f2Poly()  # 零多项式，累加用
            for k in range(n):
                prod = A[i][k] * B[k][j]  # f2Poly 乘法
                total = total + prod      # f2Poly 加法（F2 上）
            C[i][j] = total
    return C
def eliminate(mat, max_terms=0):
    def divides(a, b):
        """a divides b? a,b are exponent tuples"""
        return b[0] >= a[0] and b[1] >= a[1]

    # shallow-copy matrix into new list of f2Poly copies to avoid mutating caller
    rows = len(mat)
    cols = len(mat[0]) if rows > 0 else 0
    A = [[ f2Poly(set(mat[r][c].terms)) for c in range(cols) ] for r in range(rows)]

    # collect all monomials present (for pre-dep search)
    monos = set()
    for r in range(rows):
        for c in range(cols):
            for m in A[r][c].terms:
                monos.add(m)
    monos.add((0,0))
    monos_list = sorted(monos)

    def stage(var='y'):
        pivot_row = 0
        for c in range(cols):
            # find pivot among rows pivot_row..end with nonzero entry, choosing smallest degree (y or x)
            best_r = None
            best_deg = None
            for rr in range(pivot_row, len(A)):
                ent = A[rr][c]
                if len(ent.terms) == 0:
                    continue
                key = ent.degrees()[1] if var == 'y' else ent.degrees()[0]
                if best_r is None or key < best_deg:
                    best_r = rr
                    best_deg = key
            if best_r is None:
                continue
            # swap to pivot_row if needed
            if best_r != pivot_row:
                A[pivot_row], A[best_r] = A[best_r], A[pivot_row]
            pivot = A[pivot_row][c]
            # reduce other rows w.r.t pivot using leading monomial division
            LM = None
            for rr in range(len(A)):
                if rr == pivot_row:
                    continue
                # repeat reduction while possible (target's leading term divisible by pivot LM)
                while True:
                    target = A[rr][c]
                    if len(target.terms) == 0:
                        break
                    LM = max(pivot.terms, key=lambda m: (m[1], m[0]) if  ('y>x' if var=='y' else 'x>y') else (m[0], m[1])) if pivot.terms else None
                    if LM is None:
                        break
                    lt = max(target.terms, key=lambda m: (m[1], m[0]) if  ('y>x' if var=='y' else 'x>y') else (m[0], m[1])) if target.terms else None
                    if lt is None:
                        break
                    if divides(LM, lt):
                        mult = (lt[0] - LM[0], lt[1] - LM[1])
                        for cc in range(cols):
                            A[rr][cc] = (A[rr][cc] + A[pivot_row][cc] * f2Poly({mult}))
                        # continue trying to reduce same rr
                        continue
                    else:
                        break
            pivot_row += 1
            if pivot_row >= len(A):
                break

    # y-stage then x-stage
    stage('y')
    stage('x')

    # --- final merging: 若有纯 x 行与纯 y 行且不在同一行，则乘幂使两元素相同并消掉一行 ---
    rx = ry = None; ax = by = None; cx = cy = None

    # safer scan for monomial entries:
    rx = ry = None
    for r in range(len(A)):
        for c in range(cols):
            e = A[r][c]
            if len(e.terms) == 0:
                continue
            if len(e.terms) == 1:
                (i,j) = next(iter(e.terms))
                if i > 0 and j == 0 and rx is None:
                    rx, cx, ax = r, c, i
                if j > 0 and i == 0 and ry is None:
                    ry, cy, by = r, c, j

    if rx is not None and ry is not None and rx != ry:
        for c in range(cols):
            A[rx][c] = A[rx][c] * f2Poly({(0, by)})
            A[ry][c] = A[ry][c] * f2Poly({(ax, 0)})
        for c in range(cols):
            A[rx][c] = (A[rx][c] + A[ry][c])
        if all(len(A[rx][c].terms) == 0 for c in range(cols)):
            rowv = A.pop(rx)
            A.append(rowv)
    return A
x1=[[f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()],
    [f2Poly()]]
x2=[[f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()],
    [f2Poly()]]
z1=[[f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})],
    [f2Poly()]]
z2=[[f2Poly()],
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0):1})]]
# 示例 1: 3x + 2y → 模 2 后是 x
s1=[
    [f2Poly({(0, 0): 1,(-1,0):1})],
    [f2Poly({(0, 0): 1,(0,-1):1})],
    [f2Poly()],
    [f2Poly()]
]
s2=[
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0, 0): 1,(0,1):1})],
    [f2Poly({(0, 0): 1,(1,0):1})],
]
m=1
m1=[]
m1.extend(tdmap(excimap(s1,s2,oper=x1),m))
m1.extend(tdmap(excimap(s1,s2,oper=x2),m))
m1.extend(tdmap(excimap(s1,s2,oper=z1),m))
m1.extend(tdmap(excimap(s1,s2,oper=z2),m))
x=f2Poly({(1,0)})
y=f2Poly({(0,1)})
one=f2Poly({(0,0)})
zero=f2Poly()
ma=[[x+y,x**2+y,x+y**2],
    [x,y,zero],
    [x**2,x**3+y**2+x*y,x**-2+x*y**2]]
print(ma[0][1].degrees()[0])
print(ma[0][2].degrees())

2
(1, 2)


In [696]:
a=[[x*y**2],
   [x**2*y]]
print(eliminate(a))
print(eliminate(a)[0][0]*x+eliminate(a)[1][0]*y)

[[x^1y^2], [x^2y^1]]
0
最小值: 0, 索引: 1


In [693]:
def eli(f):
    xd=[]
    yd=[]
    de=[]
    for i in range(len(f)):
        for j in range(len(f[i])):
            de.append(f[i][j].mindegrees())
    for (i,j) in de:
        xd.append(i)
        yd.append(j)
    if min(xd)<0 or min(yd)<0:
        ff=[[c*f2Poly({(-min(xd),-min(yd))})for c in f[i]]for i in range(len(f))]
    else:
        ff=f
    for i in range(len(ff[0])):
        f1=[[ff[j][i] for j in range(len(ff))]]
        yd=[]
        for m in range(len(f1)):
            yd.extend(f1[m][0].degrees()[1])
        min_value, min_index = min((value, index) for index, value in enumerate(yd))
        xdeg,ydeg=f1[min_index][0].degrees()
        for m in range(len(ff)):
            fmdeg=ff[m][min_index].degrees()[1]
            if m==min_index:
                break
            ffdeg=ff[m][i].degrees()[1]
            deg=ffdeg-fmdeg

    return ff,de
print()

([[x^2y^1 + x^3, x^4 + x^2y^1, x^2y^2 + x^3], [x^3, x^2y^1, 0], [x^4, x^3y^1 + x^5 + x^2y^2, x^3y^2 + 1]], [(0, 0), (0, 0), (0, 0), (1, 0), (0, 1), (0, 0), (2, 0), (0, 0), (-2, 0)])
